In [169]:
import pandas as pd
import numpy as np
import random

# **1) Model creation**

In [172]:
def create_node():
    """
    Function that returns a dictionary representing a node

    feature_index: the feature of the data the node is going to split
    threhsold: the value to split
    is_final: boolean indicating whether or not the node is a leaf
    label: the label we have to return if the node is a leaf
    child_true: the tree we go on if we condition based on the feature_index and the threhsold is verified
    child_false: the tree we go otherwise
    indicies: the indicies of X_train that go through this node
    current_depth: current depth of the node in the tree

    """

    dico = {'feature_index': None,
            'threshold': None,
            'is_final': False,
            'label': None,
            'child_true': {},
            'child_false': {},
            'indicies': [],
            'current_depth': None
            }
      
    return dico


def get_proportions(y, nb_labels):
    proportions = [0 for i in range(nb_labels)]
    n = len(y)
    
    if n > 0:
        for i in range(nb_labels):
            proportions[i] = len(np.nonzero(y == i)[0]) / n

        return proportions

    else:
        return []


def compute_criterion(type='gini', y=[], nb_labels=2):
    n = len(y)

    if n > 0:
        proportions = get_proportions(y, nb_labels)
        
        res = 0
        
        for i in range(nb_labels):
            if type == 'gini':
                res += (proportions[i]*(1 - proportions[i]))

            elif type == 'entropy' or type == 'log_loss':
                if proportions[i] != 0:
                    res -= proportions[i]*np.log(proportions[i])
                    
            else:
                raise Exception("The criterion you specified does not exist or is not implemented yet")

        return res
    
    else:
        return 0

In [210]:
class DecisionTreeClassifier:
    def __init__(self, criterion='gini', max_depth=None, min_samples_split=2, max_features=None):
        self.tree = {}
        self.criterion = criterion
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_features = max_features

    def __call__(self, X):
        return self.predict(X)

    def compute_max_features(self, nb_features):
        if self.max_features == None:
            self.max_features = nb_features
            
        elif isinstance(self.max_features, int):
            try:
                assert (self.max_features > 0) and (self.max_features <= nb_features)
            except:
                raise Exception("The number of features to consider when splitting must be contained between 0 and the number of features of X_train")
        
        elif isinstance(self.max_features, float):
            self.max_features = max(1, int(self.max_features * nb_features))
            try:
                assert (self.max_features > 0) and (self.max_features <= nb_features)
            except:
                raise Exception("The ratio of features to consider when splitting must be contained between 0 and 1")
            
        elif self.max_features == "sqrt":
            self.max_features = max(1, np.floor(np.sqrt(nb_features)))
        
        elif self.max_features == "log2":
            self.max_features = max(1, np.floor(np.log2(nb_features)))
        
        elif self.max_features == "auto":
            self.max_features = max(1, np.floor(np.sqrt(nb_features)))
            
    def fit(self, X, y):
        X = X.to_numpy()
        y = y.to_numpy()

        n, nb_features = X.shape
        nb_labels = np.max(y)
        
        # Gestion de max_features
        self.compute_max_features(nb_features)

        # On initialise l'arbre à un noeud
        root = create_node()
        root['indicies'] = np.arange(n)
        root['current_depth'] = 1

        # On initialise la pile de noeuds
        to_compute = [root]

        # Tant qu'il reste un noeud à traiter
        while (to_compute):
            # On récupère le dernier élément de to_compute et on le retire
            node = to_compute.pop()

            # On récupère les lignes du dataset qui correspondent à notre noeud
            indicies = node['indicies']
            X_tronq = X[indicies]
            y_tronq = y[indicies]
            nb_samples = len(y_tronq)

            # On vérifie s'il n'y a qu'une seule classe
            if len(np.unique(y_tronq)) == 1:
                # S'il n'y a qu'une classe, le noeud est une feuille
                node['is_final'] = True
                node['label'] = y_tronq[0]
                
            elif len(indicies) < self.min_samples_split:
                label = np.argmax(get_proportions(indicies, nb_labels=nb_labels))
                node['is_final'] = True
                node['label'] = label
            
            else:
                # Sinon, s'il y a plusieurs classes
                # On stocke la valeur max du critère et les indices correspondant
                min_criterion = np.inf
                feature_index = None
                threshold = None
                found_criterion = False

                # On sélectionne les features à checker (choisies de manière aléatoire avec self.max_features)
                features_to_check = [i for i in range(nb_features)]
                random.seed(random.randint(0, 100))
                random.shuffle(features_to_check)
                features_to_check = features_to_check[:self.max_features]
                print(features_to_check)
                for i in features_to_check:
                    print("Feature : ", i)
                    # On récupère les valeurs possibles
                    possible_values = np.unique(X_tronq[:, i])
                    possible_values = np.sort(possible_values)

                    if len(possible_values) >= 2:
                        possible_values = possible_values[1:]
                        found_criterion = True
                        
                        """il faudrait optimiser le calcul des proportions pour chaque valeur, car actuellement la complexité est en O(n²)
                        alors qu'on peut faire du O(n)"""
                        # Pour chaque valeur
                        for value in possible_values:
                            print("Value : ", value)
                            # On récupère les indices où la valeur de la feature est inférieure
                            lower_indexes = np.nonzero(X_tronq[:, i] < value)[0]
                            # Et les indices où la valeur est supérierue ou égale
                            greater_indexes = np.nonzero(X_tronq[:, i] >= value)[0]

                            # On récupère les labels correspondant
                            y_lower = y_tronq[lower_indexes]
                            y_greater = y_tronq[greater_indexes]

                            # On calcule les critères
                            criterion_lower = compute_criterion(self.criterion, y_lower, nb_labels=nb_labels)
                            criterion_greater = compute_criterion(self.criterion, y_greater, nb_labels=nb_labels)

                            # On calcule le critère final
                            nb_samples_lower = len(y_lower)
                            nb_samples_greater = len(y_greater)
                           
                            criterion_split = nb_samples_lower/nb_samples*criterion_lower + nb_samples_greater/nb_samples*criterion_greater

                            if (criterion_split < min_criterion):
                                min_criterion = criterion_split
                                feature_index = i
                                threshold = value
                
                
                new_depth = node['current_depth'] + 1
                
                if found_criterion:
                    # On met à jour les informations du noeud
                    node['feature_index'] = feature_index
                    node['threshold'] = threshold
                    node['is_final'] = False
                    
                    # On récupère les indices pour le noeud où la condition est vérifiée
                    true_index = np.nonzero(X_tronq[:, feature_index] < threshold)[0]
                    # Pareil pour le noeud où la condition n'est pas vérifiée
                    false_index = np.nonzero(X_tronq[:, feature_index] >= threshold)[0]

                    # On crée les enfants du noeud, et on met leurs indices
                    child_true = create_node()
                    child_true['indicies'] = indicies[true_index]
                    child_true['current_depth'] = new_depth

                    child_false = create_node()
                    child_false['indicies'] = indicies[false_index]
                    child_false['current_depth'] = new_depth

                    node['child_true'] = child_true
                    node['child_false'] = child_false

                    # On les ajoute à la pile
                    if self.max_depth and (new_depth < self.max_depth):
                        to_compute.append(child_true)
                        to_compute.append(child_false)

                    else:
                        label_true = np.argmax(get_proportions(indicies[true_index], nb_labels=nb_labels))
                        label_false = np.argmax(get_proportions(indicies[false_index], nb_labels=nb_labels))

                        child_true['label'] = label_true
                        child_true['is_final'] = True
                        child_false['label'] = label_false
                        child_false['is_final'] = True
                        
                else:
                    label = np.argmax(get_proportions(indicies, nb_labels=nb_labels))
                    node['is_final'] = True
                    node['label'] = label
                    
        self.tree = root


    def predict(self, X):
        X = X.to_numpy()

        res = []
        for i in range(len(X)):
            res.append(self.predict_one(X[i]))
        
        return np.array(res)

    
    def predict_one(self, x):
        current_node = self.tree

        while not current_node['is_final']:
            threshold = current_node['threshold']
            feature_index = current_node['feature_index']
            if x[feature_index] < threshold:
                current_node = current_node['child_true']
            else:
                current_node = current_node['child_false']

        return current_node['label']
        

# **2) Load dataset**

Creation of a dataset that contains weight and size for adults (label 1) and children (label 0)

In [211]:
df = pd.read_csv("star_classification.csv") # Récupérer sur https://www.kaggle.com/datasets/fedesoriano/stellar-classification-dataset-sdss17
df.head()

,obj_ID,alpha,delta,u,g,r,i,z,run_ID,rerun_ID,cam_col,field_ID,spec_obj_ID,class,redshift,plate,MJD,fiber_ID
0,1.237661e+18,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,3606,301,2,79,6.543777e+18,GALAXY,0.634794,5812,56354,171
1,1.237665e+18,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,4518,301,5,119,1.176014e+19,GALAXY,0.779136,10445,58158,427
2,1.237661e+18,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,3606,301,2,120,5.152200e+18,GALAXY,0.644195,4576,55592,299
3,1.237663e+18,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,4192,301,3,214,1.030107e+19,GALAXY,0.932346,9149,58039,775
4,1.237680e+18,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,8102,301,3,137,6.891865e+18,GALAXY,0.116123,6121,56187,842


In [212]:
df = df.drop(columns=['obj_ID'])
dico = {'GALAXY': 0, 'QSO': 1, 'STAR': 2}
df['class'] = df['class'].apply(lambda s: dico[s])

# **3) Test**

In [213]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X = df.drop(columns=['class'])
y = df['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [214]:
model = DecisionTreeClassifier(criterion='gini', max_depth=1, min_samples_split=2, max_features=3)
model.fit(X_train, y_train)

[11, 8, 13]
Feature :  11
Value :  2.99590832514689e+17
Value :  2.9959358129375846e+17
Value :  2.9961227299143066e+17
Value :  2.996213439623598e+17
Value :  2.996243676193362e+17
Value :  2.996438839507292e+17
Value :  2.996455332181709e+17
Value :  2.996557037007278e+17
Value :  3.006287711222108e+17
Value :  3.006293208780247e+17
Value :  3.0064388940709274e+17
Value :  3.0069391718615654e+17
Value :  3.006950166977843e+17
Value :  3.0071453302917734e+17
Value :  3.0076208690707866e+17
Value :  3.0077088300010086e+17
Value :  3.0078984957568e+17
Value :  3.007903993314939e+17
Value :  3.019044799077683e+17
Value :  3.028835996595917e+17
Value :  3.030061952060887e+17
Value :  3.052167628807475e+17
Value :  3.0528190894469325e+17
Value :  3.0529372869469184e+17
Value :  3.06262399411841e+17
Value :  3.063652037490381e+17
Value :  3.075306863429161e+17
Value :  3.085765960406405e+17
Value :  3.108539594325627e+17
Value :  3.1199332834006835e+17
Value :  3.120131195493683e+17
Value :

Value :  3.705027425835684e+17
Value :  3.705129130661253e+17
Value :  3.705527703626322e+17
Value :  3.7055441963007386e+17
Value :  3.705590925544919e+17
Value :  3.7056046694402656e+17
Value :  3.7056431523472384e+17
Value :  3.7056568962425856e+17
Value :  3.705728364498391e+17
Value :  3.705764098626294e+17
Value :  3.7058190742076826e+17
Value :  3.7059180302541824e+17
Value :  3.705934522928599e+17
Value :  3.7059372717076685e+17
Value :  3.7156954900846394e+17
Value :  3.7160610777008736e+17
Value :  3.716269984910152e+17
Value :  3.716385433631068e+17
Value :  3.7164514043287354e+17
Value :  3.716478892119429e+17
Value :  3.716489887235706e+17
Value :  3.716514626247332e+17
Value :  3.7165503603752346e+17
Value :  3.7165805969449984e+17
Value :  3.7166685578752205e+17
Value :  3.7166795529914976e+17
Value :  3.716685050549637e+17
Value :  3.716696045665914e+17
Value :  3.7167070407821926e+17
Value :  3.717127603979817e+17
Value :  3.717157840549581e+17
Value :  3.7172155649100

Value :  3.963599514606244e+17
Value :  3.9636709828620486e+17
Value :  3.9638908851876045e+17
Value :  3.9639293680945766e+17
Value :  3.964138275303854e+17
Value :  3.964330689838714e+17
Value :  3.964465380013117e+17
Value :  3.964539597047992e+17
Value :  3.964542345827062e+17
Value :  3.96489693832702e+17
Value :  3.985743695231324e+17
Value :  3.985988336568504e+17
Value :  3.986463875347517e+17
Value :  3.986911926335836e+17
Value :  3.9871758091265024e+17
Value :  4.0308071836578e+17
Value :  4.030809932436869e+17
Value :  4.030864908018258e+17
Value :  4.030963864064758e+17
Value :  4.031642812494909e+17
Value :  4.0324537023203936e+17
Value :  4.0761125906470714e+17
Value :  4.0767283171586246e+17
Value :  4.0767420610539725e+17
Value :  4.076876751228375e+17
Value :  4.087030739433165e+17
Value :  4.087044483328512e+17
Value :  4.087047232107581e+17
Value :  4.0871489369331514e+17
Value :  4.0872396466424416e+17
Value :  4.087531017223802e+17
Value :  4.087613480595886e+17
V

Value :  4.469971365218488e+17
Value :  4.470177523648696e+17
Value :  4.470224252892877e+17
Value :  4.470389179637043e+17
Value :  4.470397425974252e+17
Value :  4.4704331601021536e+17
Value :  4.470499130799821e+17
Value :  4.4706090819625984e+17
Value :  4.4706283234160845e+17
Value :  4.4706475648695706e+17
Value :  4.470787752602112e+17
Value :  4.471131349985792e+17
Value :  4.471153340218348e+17
Value :  4.47119182312532e+17
Value :  4.4712467987067085e+17
Value :  4.471340257195069e+17
Value :  4.471343005974138e+17
Value :  4.471362247427625e+17
Value :  4.471461203474125e+17
Value :  4.471466701032264e+17
Value :  4.471546415625277e+17
Value :  4.4811149147271366e+17
Value :  4.4811286586224845e+17
Value :  4.481136904959693e+17
Value :  4.481142402517832e+17
Value :  4.481145151296901e+17
Value :  4.481169890308526e+17
Value :  4.481213870773637e+17
Value :  4.481233112227123e+17
Value :  4.4812468561224704e+17
Value :  4.4813155755992064e+17
Value :  4.4813238219364154e+17

Value :  4.8880386867150234e+17
Value :  4.897708890977997e+17
Value :  4.8988551318499526e+17
Value :  4.899077782954578e+17
Value :  4.8992921877219936e+17
Value :  4.8993004340592026e+17
Value :  4.909404946086236e+17
Value :  4.909410443644375e+17
Value :  4.9099629482373325e+17
Value :  4.910460477248901e+17
Value :  4.910487965039596e+17
Value :  4.920229636887286e+17
Value :  4.9203395880500634e+17
Value :  4.9204660318872576e+17
Value :  4.9214885777010886e+17
Value :  4.921785445840589e+17
Value :  4.921812933631283e+17
Value :  4.9218156824103526e+17
Value :  4.921906392119644e+17
Value :  4.932302275902444e+17
Value :  4.932480946541957e+17
Value :  4.9325716562512486e+17
Value :  4.9429235570523546e+17
Value :  4.942931803389562e+17
Value :  4.942975783854674e+17
Value :  4.9541605672304026e+17
Value :  4.954237533044346e+17
Value :  4.95431999641643e+17
Value :  4.954407957346652e+17
Value :  4.9550236838582074e+17
Value :  4.95510614723029e+17
Value :  4.955224344730276e+

...

Value :  4.960789029987834e+18
Value :  4.960797826080856e+18
Value :  4.960804423150623e+18
Value :  4.960807721685506e+18
Value :  4.96080827144132e+18
Value :  4.960808821197134e+18
Value :  4.960811569976203e+18
Value :  4.960812394609924e+18
Value :  4.960822565092481e+18
Value :  4.960833560208759e+18
Value :  4.960870943604103e+18
Value :  4.960871493359917e+18
Value :  4.960877265795963e+18
Value :  4.960879739697124e+18
Value :  4.960893483592473e+18
Value :  4.960895957493635e+18
Value :  4.960903104319216e+18
Value :  4.96189733814484e+18
Value :  4.962115041447139e+18
Value :  4.963186790473423e+18
Value :  4.963218401432721e+18
Value :  4.9652250151698e+18
Value :  4.965225290047707e+18
Value :  4.965262948320958e+18
Value :  4.965285488309328e+18
Value :  4.965290985867467e+18
Value :  4.965312976100022e+18
Value :  4.96531407561165e+18
Value :  4.965323421460486e+18
Value :  4.965427600187218e+18
Value :  4.965447941152332e+18
Value :  4.966352563958209e+18
Value :  4.96

Value :  5.008099635572791e+18
Value :  5.008108431665814e+18
Value :  5.008211785758824e+18
Value :  5.008264562316958e+18
Value :  5.009131252458216e+18
Value :  5.009131802214029e+18
Value :  5.009132351969843e+18
Value :  5.00913620026054e+18
Value :  5.009148020010539e+18
Value :  5.009148569766353e+18
Value :  5.009151043667515e+18
Value :  5.009151593423329e+18
Value :  5.009151868301236e+18
Value :  5.009159015126817e+18
Value :  5.009162863417514e+18
Value :  5.009166161952397e+18
Value :  5.009171109754721e+18
Value :  5.009174408289606e+18
Value :  5.009177431946582e+18
Value :  5.009177706824489e+18
Value :  5.009179081214024e+18
Value :  5.009189251696581e+18
Value :  5.009196123644254e+18
Value :  5.009197772911696e+18
Value :  5.00919832266751e+18
Value :  5.00920821827216e+18
Value :  5.009210417295415e+18
Value :  5.009210967051229e+18
Value :  5.009219213388438e+18
Value :  5.009219763144251e+18
Value :  5.00923158289425e+18
Value :  5.009239004597737e+18
Value :  5.0

Value :  5.031809234888841e+18
Value :  5.031811983667912e+18
Value :  5.031821329516747e+18
Value :  5.031829300976048e+18
Value :  5.031829575853955e+18
Value :  5.031839471458605e+18
Value :  5.031853765109766e+18
Value :  5.031861461691161e+18
Value :  5.031862561202788e+18
Value :  5.032824079490767e+18
Value :  5.032856515083785e+18
Value :  5.032863661909367e+18
Value :  5.032870808734947e+18
Value :  5.032938428700056e+18
Value :  5.032972788438424e+18
Value :  5.032980210141911e+18
Value :  5.033025564996557e+18
Value :  5.033905999435815e+18
Value :  5.033927439912557e+18
Value :  5.033953278435809e+18
Value :  5.0339568518486e+18
Value :  5.033958226238134e+18
Value :  5.034006054993944e+18
Value :  5.034006604749756e+18
Value :  5.034025846203241e+18
Value :  5.034027495470684e+18
Value :  5.03403134376138e+18
Value :  5.034032443273009e+18
Value :  5.034033542784637e+18
Value :  5.034034092540451e+18
Value :  5.034046736924169e+18
Value :  5.034057732040448e+18
Value :  5.

Value :  5.053271696964082e+18
Value :  5.053273346231523e+18
Value :  5.05327994330129e+18
Value :  5.053284341347801e+18
Value :  5.053284616225708e+18
Value :  5.053285990615243e+18
Value :  5.053291213295475e+18
Value :  5.053298085243148e+18
Value :  5.053300284266404e+18
Value :  5.053301383778032e+18
Value :  5.053302483289659e+18
Value :  5.054165874359163e+18
Value :  5.054168898016139e+18
Value :  5.054169447771953e+18
Value :  5.054186215324277e+18
Value :  5.054213703114971e+18
Value :  5.054214802626599e+18
Value :  5.054242840173107e+18
Value :  5.054243664806828e+18
Value :  5.054265380161477e+18
Value :  5.054287095516125e+18
Value :  5.05431183452775e+18
Value :  5.05432805232426e+18
Value :  5.054329426713795e+18
Value :  5.054349217923096e+18
Value :  5.054357464260303e+18
Value :  5.054421785690528e+18
Value :  5.055291775675292e+18
Value :  5.055292325431106e+18
Value :  5.055295898843896e+18
Value :  5.055301946157849e+18
Value :  5.055318163954358e+18
Value :  5.

KeyboardInterrupt: 

In [196]:
preds = model.predict(X_train)
print("Training score : ", accuracy_score(preds, y_train))

preds = model.predict(X_test)
print("Testing score : ", accuracy_score(preds, y_test))

0.9814814814814815
0.42857142857142855
